# Base de datos Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Conexion a RIAK

In [2]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

## Funciones utiles para el desarrollo de la práctica

In [3]:
# ELimina datos de un Bucket
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        # print(keys)
        for keys in bucket.stream_keys():
            for key in keys:
                bucket.delete(key)

In [4]:
# Devuelve el hash de un string
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

In [5]:
# Convertimos el objeto JSON en un objeto pandas 
def json_to_pandas(jsonObj):
    return pd.read_json(json.dumps(jsonObj))

## Lectura de la informacion en Pandas

In [6]:
df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

In [7]:
# 'merge' del tipo de encadenamiento en el data frame de ascensos
df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = df_ascensos[['id_escalador','id_dificultad', 'tipo_encadenamiento', 'nombre_via', 'risco', 'sector', 'pais']];


## Desnormalización del data frame de ascensos

Voy a incluir la información del usuario en cada registro de ascenso, para ir creando los buckets que van interesando según las preguntas a resolver


In [8]:
df_ascensos_desnormalizado = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');
df_ascensos_desnormalizado = pd.merge(df_ascensos_desnormalizado, df_dificultades, on = ['id_dificultad'], how = 'inner');

#renombrar campos autonombrados por merge
df_ascensos_desnormalizado = df_ascensos_desnormalizado.rename(
    columns={
        'pais_x' : 'pais_risco',
        'ciudad':'ciudad_escalador',
        'pais_y':'pais_escalador'});

df_ascensos_desnormalizado.head()

,id_escalador,id_dificultad,tipo_encadenamiento,nombre_via,risco,sector,pais_risco,nombre,sexo,fecha_nacimiento,ciudad_escalador,pais_escalador,comienzo,grado_frances,grado_usa,grado_bloque_usa
0,66274,25,Toprope,KVARTETTEN,LASSEHAGA,BôNN,SWE,Sara Herrmann,Hombre,2555-12-31,Lysekil,SWE,0,4c,5.6,V0
1,29844,25,Toprope,"FORGIVE ME AMY, FOR I HAVE SENT",RED RIVER GORGE,GOLD COAST,USA,Tom Burton,Hombre,1962-04-21,Ogden,USA,1992,4c,5.6,V0
2,56407,25,Toprope,EL TIó LILA,MONT-RAL,TERRANEGRA,ESP,Susana Barrera Vilarmau,Mujer,1978-01-19,Manresa,ESP,0,4c,5.6,V0
3,66273,25,Toprope,GUNNENG,KOLSåS,ØVRE SYDSTUP,NOR,Erling Fasting,Hombre,1988-01-08,Trondheim,NOR,2016,4c,5.6,V0
4,65841,25,Toprope,MYRZEBIAKI,DOLINA SZKLARKI,SLONECZNE SKALY,POL,Justyna Wosik,Mujer,2555-12-31,Sosnowiec,POL,0,4c,5.6,V0


## PREPARACIÓN DE LA BASE DE DATOS

Para resolver las preguntas me interesará tener los siguientes buckets:

|bucket|tipo|clave|Contenido del bucket|
|-|-|-|
|dificultades|default|id_dificultad|dificicultad en grado frances, usa y bloque-usa|
|ascensos|default|generada con identificador unico|Información desnormalizada de cada ascenso|
|ascensos_acum|'map'|id_escalador|Un mapa por escalador con: nombre, sexo, pais, ascensos_acumulados, suma_dificultades, , dificultad maxima en españa, ascensos acumulados en españa y suma dificultad en España|
|riscos_acum|'map'|hash del nombre del risco|Un mapa por risco: nombre, pais, suma_ascensos, suma_dificultades, dificultad maxima|

** Indices por bucket **

|Bucket|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|-|
|ascensos|idx_IdEscalador_int|Id del escalador|Localizar movimientos de un escalador concreto|



**CREACIÓN DE JSON CON LA INFORMACIÓN QUE IRÁ A LOS BUCKETS**

In [9]:
ascensosJsonString = df_ascensos_desnormalizado.to_json(orient = 'records')
ascensosJson = json.loads(ascensosJsonString)

In [10]:
dificultadesJsonString = df_dificultades.to_json(orient = 'records')
dificultadesJson = json.loads(dificultadesJsonString)

**CREACIÓN DE BUCKET ASCENSOS ACUMULADOS**

In [11]:
BUCKET_ACUM_ASCENSOS = 'acum_ascensos'
bucket_acum_ascensos = cliente.bucket_type('maps').bucket(BUCKET_ACUM_ASCENSOS)
drop_keys(bucket_acum_ascensos)



for ascensoJson in ascensosJson:
    # construcción de ascensos acumulados por escalador
    acumulador_ascensos = bucket_acum_ascensos.new(str(ascensoJson['id_escalador']))
    acumulador_ascensos.registers['nombre'].assign(ascensoJson['nombre'].encode('utf-8'))
    acumulador_ascensos.registers['sexo'].assign(ascensoJson['sexo'].encode('utf-8'))
    acumulador_ascensos.registers['pais'].assign(ascensoJson['pais_escalador'].encode('utf-8'))
    acumulador_ascensos.counters['numero_ascensos'].increment()      
    acumulador_ascensos.counters['suma_dificultades'].increment(ascensoJson['id_dificultad'])
    
    if (ascensoJson['pais_risco'] == 'ESP') :
        
        str_dificultad_maxima_registrada = acumulador_ascensos.registers['id_dificultad_max'].value  
        dificultad_maxima = ascensoJson['id_dificultad']
        
        if (str_dificultad_maxima_registrada != ''):
            dificultad_maxima = max(int(str_dificultad_maxima_registrada), ascensoJson['id_dificultad'])
            
        acumulador_ascensos.registers['id_dificultad_max_esp'].assign(str(dificultad_maxima))
        acumulador_ascensos.counters['numero_ascensos_esp'].increment()
        acumulador_ascensos.counters['suma_dificultades_esp'].increment(ascensoJson['id_dificultad'])
        
    acumulador_ascensos.store()
    
    
    

**CREACIÓN DE BUCKET RISCOS ACUMULADOS**

In [12]:
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

In [13]:
BUCKET_ACUM_RISCOS = 'acum_riscos'
bucket_acum_riscos = cliente.bucket_type('maps').bucket(BUCKET_ACUM_RISCOS)
drop_keys(bucket_acum_riscos)


for ascensoJson in ascensosJson:
    acumulador_risco = bucket_acum_riscos.new(hash_string(ascensoJson['risco']))
    
    str_dificultad_maxima_registrada = acumulador_risco.registers['id_dificultad_max'].value
    dificultad_maxima = ascensoJson['id_dificultad']
    if (str_dificultad_maxima_registrada != ''):
        dificultad_maxima = max(int(str_dificultad_maxima_registrada), ascensoJson['id_dificultad'])
    
    acumulador_risco.registers['risco'].assign(ascensoJson['risco'].encode('utf-8'))
    acumulador_risco.registers['pais'].assign(ascensoJson['pais_risco'].encode('utf-8'))
    acumulador_risco.registers['id_dificultad_max'].assign(str(dificultad_maxima))
        
    acumulador_risco.counters['numero_ascensos'].increment()
    acumulador_risco.counters['suma_dificultades'].increment(ascensoJson['id_dificultad'])
        
    acumulador_risco.store()

**CREACIÓN DE BUCKET ASCENSOS**

In [14]:
BUCKET_ASCENSOS = 'ascensos'
bucket_ascensos = cliente.bucket(BUCKET_ASCENSOS)
drop_keys(bucket_ascensos)

for ascensoJson in ascensosJson:
    key = uuid.uuid1().hex
    ascenso = bucket_ascensos.new(key, ascensoJson)
    ascenso.add_index('idx_IdEscalador_int', ascensoJson["id_escalador"])
    ascenso.store()

**CREACIÓN DE BUCKET DIFICULTADES**

In [15]:
BUCKET_DIFICULTADES = 'dificultades'
bucket_dificultades = cliente.bucket(BUCKET_DIFICULTADES)
drop_keys(bucket_dificultades)

for dificultadJson in dificultadesJson:
    dificultad = bucket_dificultades.new(str(dificultadJson["id_dificultad"]), dificultadJson)
    dificultad.store()

## LECTURA DE LOS BUCKETS RIAK

**DATA FRAME RISCOS ACUMULADOS**

In [16]:
listaJson = []
for streamKeys in bucket_acum_riscos.stream_keys():
    for streamKey in streamKeys:
        acumulador_riscos = bucket_acum_riscos.get(streamKey)
        listaJson.append({'risco' : acumulador_riscos.registers['risco'].value,
                          'pais' : acumulador_riscos.registers['pais'].value,
                          'id_dificultad_max' : acumulador_riscos.registers['id_dificultad_max'].value,
                          'numero_ascensos' : acumulador_riscos.counters['numero_ascensos'].value,
                          'suma_dificultades' : acumulador_riscos.counters['suma_dificultades'].value})
        
# Convertimos el objeto json en un objeto pandas 
df_bucket_riscos_acumulados = json_to_pandas(listaJson)

La columna de identificacion de dificultad maxima viene de un registro por lo que es un string. Debemos convertirlo en una columna de enteros:

In [17]:
df_bucket_riscos_acumulados['id_dificultad_max'] = pd.to_numeric(df_bucket_riscos_acumulados['id_dificultad_max'])
df_bucket_riscos_acumulados['id_dificultad_max'].fillna(value=0, inplace =True)
df_bucket_riscos_acumulados['id_dificultad_max'] = df_bucket_riscos_acumulados['id_dificultad_max'].astype(int)
df_bucket_riscos_acumulados.head(10)

,id_dificultad_max,numero_ascensos,pais,risco,suma_dificultades
0,49,1,ITA,SASSOFORTINO,49
1,49,1,ESP,POZO VERDE,49
2,40,1,SRB,VRMDZA,40
3,64,6,FRA,SEYNES,331
4,33,1,ITA,GIAGLIONE,33
5,62,3,ITA,GRESSONEY,170
6,53,2,ESP,PEDRIZA,102
7,51,2,NOR,HIMMELDALEN,82
8,57,7,DEU,BLAUTAL,257
9,40,3,USA,CATOCTIN,105


**DATA FRAME ASCENSOS ACUMULADOS**

In [18]:
listaJson = []
for streamKeys in bucket_acum_ascensos.stream_keys():
    for streamKey in streamKeys:
        acumulador_ascensos = bucket_acum_ascensos.get(streamKey)
        listaJson.append({'id_escalador' : streamKey,
                          'nombre' : acumulador_ascensos.registers['nombre'].value,
                          'sexo' : acumulador_ascensos.registers['sexo'].value,
                          'pais' : acumulador_ascensos.registers['pais'].value,
                          'numero_ascensos' : acumulador_ascensos.counters['numero_ascensos'].value,
                          'suma_dificultades' : acumulador_ascensos.counters['suma_dificultades'].value,
                          'numero_ascensos_esp' : acumulador_ascensos.counters['numero_ascensos_esp'].value,
                          'suma_dificultades_esp' : acumulador_ascensos.counters['suma_dificultades_esp'].value,
                          'id_dificultad_max_esp' : acumulador_ascensos.registers['id_dificultad_max_esp'].value})
        
# Convertimos el objeto json en un objeto pandas 
df_bucket_ascensos_acumulados = json_to_pandas(listaJson)


La columna de identificacion de dificultad maxima viene de un registro por lo que es un string. Debemos convertirlo en una columna de enteros:

In [19]:
df_bucket_ascensos_acumulados['id_dificultad_max_esp'] = pd.to_numeric(df_bucket_ascensos_acumulados['id_dificultad_max_esp'])
df_bucket_ascensos_acumulados['id_dificultad_max_esp'].fillna(value=0, inplace =True)
df_bucket_ascensos_acumulados['id_dificultad_max_esp'] = df_bucket_ascensos_acumulados['id_dificultad_max_esp'].astype(int)
#df_bucket_ascensos_acumulados.head(10)

**DATA FRAME DIFICULTADES**

In [20]:
filasDificultades = []
for keys in bucket_dificultades.stream_keys():
    for key in keys:
        filasDificultades.append(bucket_dificultades.get(key).data) 
df_bucket_dificultades = json_to_pandas(filasDificultades);
df_bucket_dificultades.head(3)

,grado_bloque_usa,grado_frances,grado_usa,id_dificultad
0,V16/17,8c+/9a,5.14c/d,73
1,V7,7a+,5.12a,51
2,V2,6,5.9,35


Como se observa, RIAK no devuelve orden, pero en nuestro caso el orden de dificultad viene dado por id_dificultad. Para facilitar posteriores mapeos, es interesante ordenar este data frame y restablecer el indice

In [21]:
df_bucket_dificultades = df_bucket_dificultades.sort_values('id_dificultad',ascending=True);
df_bucket_dificultades=df_bucket_dificultades.set_index('id_dificultad');
df_bucket_dificultades.head(3)

,grado_bloque_usa,grado_frances,grado_usa
id_dificultad,,,
1,VB,-,3/4
2,VB,1,3/4
3,VB,1a,3/4


## RESPUESTAS A LAS PREGUNTAS

### 1.a) Los 10 escaladores (hombres) más activos. (orden auxiliar por Id)

In [22]:
df = df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['sexo']=='Hombre'];
df = df.sort_values(['numero_ascensos', 'id_escalador'],ascending=[False, False]);
df[['id_escalador', 'nombre', 'sexo', 'pais', 'numero_ascensos']].head(10)

,id_escalador,nombre,sexo,pais,numero_ascensos
2105,50884,Christopher Leonetti,Hombre,USA,43
2698,20095,Matthias Schuster,Hombre,DEU,33
4565,9171,Laurenz Trawnicek,Hombre,AUT,33
80,20384,Clemens Kurth,Hombre,DEU,24
1076,23550,Moritz Perwitzschky,Hombre,DEU,22
1452,62639,Artur Gryt,Hombre,POL,21
2365,30439,Nuno Henriques,Hombre,PRT,20
3278,37830,Rodrigo Pessoa,Hombre,BRA,19
2568,60616,Alex Garriga,Hombre,ESP,17
2745,40521,Brandon Gezel,Hombre,USA,17


### 1.b) Los 10 escaladoras (mujeres) más activas. (orden auxiliar por Id)

In [23]:
df = df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['sexo']=='Mujer'];
df = df.sort_values(['numero_ascensos', 'id_escalador'],ascending=[False, False]);
df[['id_escalador', 'nombre', 'sexo', 'pais', 'numero_ascensos']].head(10)

,id_escalador,nombre,sexo,pais,numero_ascensos
1272,53983,Daniela Bärtschi,Mujer,CHE,16
2085,54695,Elfi Hasler,Mujer,AUT,13
3057,65502,Ksenia Targosz,Mujer,POL,12
1808,51142,Isabelle Kölle,Mujer,AUT,12
525,44050,Betka Galicic,Mujer,SVN,12
731,37559,Magdalena Trzemzalska,Mujer,POL,11
2571,60268,Annemarie Van den Berg,Mujer,NLD,10
3115,55467,Katrin Gründler,Mujer,DEU,10
3864,49469,Line Tveter,Mujer,NOR,10
2892,22291,Stella Mascari,Mujer,USA,10


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de vía ascendete)

En este caso voy a obtener los **ascensos de la escaladora más activa** (de la pregunta anterior se que id_escalador=53983) **accediendo por el indice al bucket de ascensos**:

In [24]:
filasAscensos = []
streamIndex = bucket_ascensos.stream_index("idx_IdEscalador_int", 53983)
for keys in streamIndex.results:
    for key in keys:
        filasAscensos.append(bucket_ascensos.get(key).data)
    
df_bucket_ascensos = json_to_pandas(filasAscensos);

In [25]:
df_bucket_ascensos = df_bucket_ascensos[df_bucket_ascensos['tipo_encadenamiento']=='Onsight']
df_bucket_ascensos = df_bucket_ascensos.sort_values(['id_dificultad', 'nombre_via'],ascending=[False, True])
df_bucket_ascensos[['nombre_via', 'grado_frances', 'risco', 'pais_risco']].head(10)


,nombre_via,grado_frances,risco,pais_risco
9,UNDA NIEDDA,6b+,GUTTURU CARDAXIUS,ITA
3,TIRO GHIRO,6a+,BRONTALLO,CHE
7,BRIDGE,6a,CALA FUILI,ITA
8,DOLCE EDERA,6a,ISELLE,CHE
15,MATèRIA FOSCA,6a,SIURANA,ESP
10,QUIDAM,5c,JOLIMONT,CHE
0,MATEDO,5b,SAN NICOLò,ITA
14,MOIJE.COM,5b,LE RIF D'ORIOL,FRA
6,BOLA DE DRAP,5a,ARBOLI,ESP


### 3. Dificultad media de los ascensos del escalador más activo

De la pregunta 1 ya tenemos un data frame con los datos acumulados por escalador. Utilizandolo podemos obtener la media de los ascensos del escalador más activo (id_escalador = 50884):

In [26]:
df=df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['id_escalador']==50884]
id_dificultad_media = int(round(df['suma_dificultades']/df['numero_ascensos'],0))

print('Dificultad media de ascensos escalador mas activo:')
df_bucket_dificultades['grado_frances'][id_dificultad_media]


Dificultad media de ascensos escalador mas activo:


u'6'

### 4.a) Los 10 ascensos mas dificiles

**Esta pregunta no la podemos respoder con la base de datos**. No obstante, si podemos hacerla mediante un proceso de python. La pega que tiene es que debemos construir un data frame con todos los ascensos, lo que puede suponer traer mucha información desde la base de datos. Una posible solución es paralelizar la clasificación en trozos y despues unirlos.

El proceso de python para responder esta pregunta es:

In [27]:
filasAscensos = []

for keys in bucket_ascensos.stream_keys():
    for key in keys:
        filasAscensos.append(bucket_ascensos.get(key).data)
    
df_bucket_ascensos = json_to_pandas(filasAscensos);

In [28]:
df_bucket_ascensos = df_bucket_ascensos.sort_values(['id_dificultad', 'id_escalador'], ascending=[False, True]);
columnas = ['id_escalador','nombre','pais_escalador','nombre_via','grado_frances','tipo_encadenamiento','risco','pais_risco'];
df_bucket_ascensos[columnas].head(10)

,id_escalador,nombre,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
11210,1476,Adam Ondra,CZE,SILENCE,9c,Redpoint,FLATANGER,NOR
11452,1476,Adam Ondra,CZE,MOVE HARD,9b,Redpoint,FLATANGER,NOR
11601,22437,Stefano Ghisolfi,ITA,FIRST ROUND FIRST MINUTE,9b,Redpoint,MARGALEF,ESP
10729,1476,Adam Ondra,CZE,NATURALMENTE,9a+,Redpoint,CAMAIORE,ITA
11408,1476,Adam Ondra,CZE,ULTIMATUM,9a+,Redpoint,ARCO,ITA
11239,8707,Daniel Fuertes,ESP,NO PAIN NO GAIN,9a+,Redpoint,RODELLAR,ESP
11705,14130,David Firnenburg,DEU,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
9359,18008,Piotr Schab,POL,THOR'S HAMMER,9a+,Redpoint,FLATANGER,NOR
7201,22437,Stefano Ghisolfi,ITA,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
10117,22437,Stefano Ghisolfi,ITA,ONE PUNCH,9a+,Redpoint,ARCO,ITA


### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

Aprovechando el data frame ordenado anterior, si hacemos uno nuevo cuyas filas tengan los elementos con tipo_encadenamiento igual a Onsight, hemos respondido a esta pregunta facilmente:

In [29]:
df_ascensos_onsight = df_bucket_ascensos[df_bucket_ascensos['tipo_encadenamiento']=='Onsight']
df_ascensos_onsight[columnas].head(10)

,id_escalador,nombre,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
11330,6726,Manu Lopez,FRA,WHAT,8b,Onsight,LEONIDIO,GRC
10654,10806,Guillaume Lebret,FRA,WHAT,8b,Onsight,LEONIDIO,GRC
10736,17981,Stéphane Hanssens,BEL,NEGOCIEE,8b,Onsight,SIURANA,ESP
10347,8345,Mathieu Bouyoud,FRA,LAGUNAS MENTALES,8a+,Onsight,OTIñAR,ESP
11511,14130,David Firnenburg,DEU,ZONA 0,8a+,Onsight,SIURANA,ESP
9656,31370,Stefano Carnati,ITA,ER BUS,8a+,Onsight,TRENTO,ITA
9833,34114,jose luis palao,ESP,HUMPHREY,8a+,Onsight,BIELSA,ESP
9942,46646,Hernan Garcia,MEX,EL MAL PAS,8a+,Onsight,CALDERS,ESP
11512,47324,Jonas Junker,DEU,LAS CLOCHAS DE LA TARGA,8a+,Onsight,CHULILLA,ESP
10758,3393,Jose Agustí,ESP,EL ZUMBAR SE VA A ACABAR,8a,Onsight,MARGALEF,ESP


### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

La base de datos no da funcionalidad para resolver esta pregunta de manera directa. Para responder a la pregunta, o se procesan los ascensos del bucket de ascensos o se registra a priori información en el bucket de ascensos acumulados que permita calculos posteriore.

En este caso yo he elegido registrar información en el bucket de ascensos acumulados la información necesaria ello se debe haber registrado en el bucket de acumulados informacion a priori acerca del numero de vias escaladas en España por los escaladores. En este caso, esa información la he construido en la creación del bucket anteriormente

In [30]:
#del data frame de ascensos acumulados me quedo solo con los escaladores que tienen ascensos en españa
df = df_bucket_ascensos_acumulados[df_bucket_ascensos_acumulados['numero_ascensos_esp']>0]
df['id_dificultad_media_esp'] = (df['suma_dificultades_esp']/df['numero_ascensos_esp']).round().astype(int);

df['grado_medio_esp']=df['id_dificultad_media_esp'].map(df_bucket_dificultades['grado_frances']);

df['grado_maximo_esp']=df['id_dificultad_max_esp'].map(df_bucket_dificultades['grado_frances']);

#ordeno el data frame del bucket de acumulados por numero de ascensos y por nombre
df = df.sort_values(['numero_ascensos_esp', 'id_escalador'], ascending=[False, True]);


In [31]:
columnas = ['nombre', 'pais', 'numero_ascensos_esp', 'grado_medio_esp', 'grado_maximo_esp']
df_NO_ESP = df[df['pais']!='ESP'];
df_NO_ESP[columnas].head(10)

,nombre,pais,numero_ascensos_esp,grado_medio_esp,grado_maximo_esp
2365,Nuno Henriques,PRT,20,6a+,6c+
2225,"Grzegorz ""Buła"" Golowczyk",POL,10,7b+,8b
1808,Isabelle Kölle,AUT,9,7c+,8b+
3,Steve Crowe,GBR,6,7c/+,8a
3509,Stefan Madej,POL,6,7c+/8a,8a
1918,Rajko Zajc,SVN,6,7b/+,7c
4544,Markus Glück,DEU,6,6b+/6c,7a
2986,Mickaël S,FRA,6,7c,8a
3100,Jessica Verbeek,NLD,6,5b,4b
3072,Benjamin Thomas,FRA,6,7a+,8a


### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

In [32]:
columnas = ['nombre', 'pais', 'numero_ascensos_esp', 'grado_medio_esp', 'grado_maximo_esp']
df_ESP = df[df['pais']=='ESP'];
df_ESP[columnas].head(10)

,nombre,pais,numero_ascensos_esp,grado_medio_esp,grado_maximo_esp
2568,Alex Garriga,ESP,17,7c,8b
1014,Raúl Crespo,ESP,11,6b+/6c,7b
3982,Chaken Gómez conde,ESP,11,7c/+,8a
3573,Daniel Fuertes,ESP,10,8a+,9a+
1138,Tomata Tomata,ESP,10,6c+,7b
1051,Renaud Moulin,ESP,9,7c+/8a,8a
4338,Padin Peinch,ESP,9,6a/+,6b
1772,vicen Carrasco,ESP,9,6a+,6b
1454,patrick abellaneda,ESP,9,6a/+,7b+
4424,Adrian Alameda,ESP,9,6c,8a+


### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

Estas dos preguntas **no se pueden responder de manera directa con la base de datos RIAK**. Requiere procesamiento, ya sea previo a la hora de introducir la información en la base de datos, ya sea a la hora de llevar información a la base de datos como después de su obtención.

Una **posible solución alternativa a trabajar intensivamente en el lado de python con el Data Frame de todos los datos de ascensos** podría ser:

- Plantear dos nuevos buckets tipo mapa con acumuladores, uno para los escaladores de menos de 3 años y otro para los escaladores de entre 10 y 30 años de experiencia.

- En cada bucket tener por cada escalador, registros de su información y acumuladores que permitan llevar el total de ascensos no TopRope asi como la suma de las dificultades de estos ascensos.

- Rellenariamos esos buckets recorriendo el DataFrame como hemos hecho anteriormente.

- Con esos dos nuevos buckets hariamos un proceso similar al realizado en las anteriores preguntas:
    - Leeriamos el contenido del bucket llevandolo a un pandas
    - Calculariamos las medias con los valores acumulados de los buckets
    - Ordenariamos y filtrariamos en base a los requerimientos de la pregunta.



### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

Utilizando el data frame obtenido del bucket de riscos_acumulados:

In [33]:
df_bucket_riscos_acumulados = df_bucket_riscos_acumulados.sort_values(['numero_ascensos', 'risco'], ascending=[False,True])

df_riscos_esp = df_bucket_riscos_acumulados[df_bucket_riscos_acumulados['pais']=='ESP'];

columnas = ['risco', 'numero_ascensos']
df_riscos_esp.head(10)[columnas]

,risco,numero_ascensos
1871,MARGALEF,200
841,CHULILLA,164
300,SIURANA,148
641,RODELLAR,143
1138,CUENCA,127
118,ALBARRACíN,92
249,MALLORCA,88
1668,LA PEDRIZA,54
366,MONTSERRAT,54
842,TEVERGA,40


### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

Con el data frame obtenido del bucket de riscos acumulados podemos construir esta informacion media que se solicita. El procesamiento de construccion de la media, seleccion y filtrado se debe hacer fuera ya que no es funcionalidad que de la base de datos:

In [34]:
df_riscos_esp['id_dificultad_media'] = (df_riscos_esp['suma_dificultades']/df_riscos_esp['numero_ascensos']).round().astype(int);
df_riscos_esp['dificultad_media']=df_riscos_esp['id_dificultad_media'].map(df_bucket_dificultades['grado_frances']);
#df_riscos_esp.head()

In [35]:
columnas = ['risco', 'id_dificultad_media', 'dificultad_media', 'numero_ascensos']
df_riscos_esp.sort_values(['id_dificultad_media', 'numero_ascensos', 'risco'], ascending=[False, False, True]).head(10)[columnas]

,risco,id_dificultad_media,dificultad_media,numero_ascensos
1455,LA COVA DE L'OCELL,75,9a,1
1568,TARBENA,72,8c+,1
997,SANTA LINYA,65,8a+/8b,13
370,SANTUARIO,64,8a+,2
1181,BENTUE,64,8a+,1
522,MOIA,64,8a+,1
291,POLORIA,64,8a+,1
1838,VILLANUEVA DEL ROSARIO,62,8a,9
656,CUEVA BOLLU,62,8a,1
721,FIGOLS,62,8a,1


### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente

In [36]:
df_riscos_esp.sort_values(['id_dificultad_media', 'numero_ascensos', 'risco'], ascending=[True, False, True]).head(10)[columnas]

,risco,id_dificultad_media,dificultad_media,numero_ascensos
1654,EL BUBóN,21,4a,1
957,PICOS DE EUROPA,21,4a,1
593,SUBIRATS,23,4b,1
631,BAROñA,25,4c,1
935,TIVISSA,25,4c,1
843,BALNEARIO DE PANTICOSA,27,4+,2
1833,LOBADIZ,29,5a,2
1944,BARRUECOS,29,5a,1
1689,SANT VICENç DE CASTELLET,29,5a,1
1105,TEBA,29,5a,1
